# Test project
Suma de vectores con argumentos por referencia (entradas y salidas escalares y vectores)
Notar que los datos son de tipo flotante, pero el driver lo ignora.

In [16]:
from pynq import Overlay

In [46]:
overlay = Overlay('/home/xilinx/pynq/overlays/test/test2.bit')
overlay?

In [47]:
vadd_ip = overlay.vadd_0
vadd_ip?

In [19]:
vadd_ip.register_map

RegisterMap {
  CTRL = Register(AP_START=0, AP_DONE=0, AP_IDLE=1, AP_READY=0, RESERVED_1=0, AUTO_RESTART=0, RESERVED_2=0, INTERRUPT=0, RESERVED_3=0),
  GIER = Register(Enable=0, RESERVED=0),
  IP_IER = Register(CHAN0_INT_EN=0, CHAN1_INT_EN=0, RESERVED_0=0),
  IP_ISR = Register(CHAN0_INT_ST=0, CHAN1_INT_ST=0, RESERVED_0=0),
  A = Register(A=write-only),
  Memory_B = Register(value=0),
  Memory_C = Register(value=0),
  Memory_D = Register(value=0),
  R = Register(R=0),
  R_ctrl = Register(R_ap_vld=0, RESERVED=0)
}

In [28]:
vadd_ip.write(0x10, 0x5555ffff) #A
vadd_ip.write(0x20, 0x5555afff) # B0
vadd_ip.write(0x24, 0x5555bfff) # B1
vadd_ip.write(0x28, 0x5555cfff) # B2
vadd_ip.write(0x2c, 0x5555ffff) # B3
vadd_ip.write(0x30, 0) # C0
vadd_ip.write(0x34, 0) # C1
vadd_ip.write(0x38, 0) # C2
vadd_ip.write(0x3c, 0) # C3

In [29]:
vadd_ip.write(0x00, 1)
vadd_ip.read(0x00)

14

In [30]:
vadd_ip.read(0x00)

4

In [31]:
vadd_ip.read(0x54)

1

In [32]:
vadd_ip.read(0x50)

1798497278


### Driver especializado

In [2]:
from pynq import DefaultIP
import struct

class MyDriver(DefaultIP):
    
    # IP Register Map
    CTRL_REG = 0x0000
    GIE_REG = 0x0004
    INTE_REG = 0x0008
    INTS_REG = 0x000C
    A_REG = 0x0010
    R_REG = 0x0050
    R_CTRL_REG = 0x0054
    B_REG = 0x0020
    C_REG = 0x0030
    D_REG = 0x0040
    
    base_offset = [0x0010, 0x0020, 0x0030]

     # MORE CONSTANTS
    BRAM_SIZE = 16
    BRAM_BLOCKS = 8
    
    def __init__(self, description):
        super().__init__(description=description)
        self.time_start = 0
        self.time_stop = 0

    bindto = ['xilinx.com:hls:vadd:1.0']

    def is_ready(self):
        state = self.read(MyDriver.CTRL_REG)
        return (state >> 3) & 0x01

    def is_done(self):
        state = self.read(MyDriver.CTRL_REG)
        return (state >> 1) & 0x01

    def is_idle(self):
        state = self.read(MyDriver.CTRL_REG)
        return (state >> 2) & 0x01

    def start(self):
        self.write(MyDriver.CTRL_REG, 0x00000001)


    def write_brams(self, data):
        # START TIMING
        #self.time_start = time.time_ns()
        #self.time_start = time.perf_counter_ns()
        for i in range(0,3):
            data_bytes = map(lambda x : struct.pack("f", x), data[i])
            for idx, word_item in enumerate(data_bytes):
                print(f"ADDRESS     : {hex(MyDriver.base_offset[i] + (idx * 4))}")
                print(f"DATA (float): {data[i][idx]}")
                print(f"DATA (hex)  : 0x{word_item.hex()}")
                print(f"DATA (uint) : {int.from_bytes(word_item, 'little')}")
                self.write(MyDriver.base_offset[i] + (idx * 4), int.from_bytes(word_item,'little'))


    def read_result(self):
        #STOP TIMING
        #self.time_stop = time.time_ns()
        #self.time_stop = time.perf_counter_ns()
        if (self.read(MyDriver.R_CTRL_REG)):
            R_out = self.read(MyDriver.R_REG)
            print(R_out)
            for idx in range(0,4):
                data_out = self.read(MyDriver.D_REG + (idx * 4))
                print(data_out)
            #return tuple(map(lambda x : struct.unpack("f", x.to_bytes(4,'little'))[0], data_out))
        #return (0, 1)

    def get_extime(self):
        return self.time_stop-self.time_start

In [3]:
from pynq import Overlay
overlay = Overlay('/home/xilinx/pynq/overlays/test/test2.bit')
vadd_ip = overlay.vadd_0

In [4]:
data = [[2.0],[8.0, 2.0, 3.0, 4.0],[0.0, 0.0, 0.0, 0.0]]

In [5]:
vadd_ip.write_brams(data)

ADDRESS     : 0x10
DATA (float): 2.0
DATA (hex)  : 0x00000040
DATA (uint) : 1073741824
ADDRESS     : 0x20
DATA (float): 8.0
DATA (hex)  : 0x00000041
DATA (uint) : 1090519040
ADDRESS     : 0x24
DATA (float): 2.0
DATA (hex)  : 0x00000040
DATA (uint) : 1073741824
ADDRESS     : 0x28
DATA (float): 3.0
DATA (hex)  : 0x00004040
DATA (uint) : 1077936128
ADDRESS     : 0x2c
DATA (float): 4.0
DATA (hex)  : 0x00008040
DATA (uint) : 1082130432
ADDRESS     : 0x30
DATA (float): 0.0
DATA (hex)  : 0x00000000
DATA (uint) : 0
ADDRESS     : 0x34
DATA (float): 0.0
DATA (hex)  : 0x00000000
DATA (uint) : 0
ADDRESS     : 0x38
DATA (float): 0.0
DATA (hex)  : 0x00000000
DATA (uint) : 0
ADDRESS     : 0x3c
DATA (float): 0.0
DATA (hex)  : 0x00000000
DATA (uint) : 0


In [6]:
vadd_ip.start()

In [7]:
vadd_ip.read_result()

1090519040
1090519040
1073741824
1077936128
1082130432
